In [1]:
# https://www.rev.com/blog/transcript-tag/coronavirus-update-transcripts

In [28]:
import pandas as pd
import time
import re
import requests
from datetime import datetime
from bs4 import BeautifulSoup as bs

In [3]:
briefing_response = requests.get("https://www.rev.com/blog/transcripts/donald-trump-coronavirus-briefing-transcript-april-3-new-cdc-face-mask-recommendations")

In [4]:
briefing = bs(briefing_response.text)

In [6]:
briefing_text = briefing.find("div",{"class":"fl-callout-text"})

In [46]:
# empty lists to store transcript columns
speaker = []
timestamp = []
text = []

# iterate over transcript paragraphs
for paragraph in briefing_text.find_all("p"):
    
    name_match = re.search('(.+?):', paragraph.text)
    if name_match:
        speaker.append(name_match.group(1))

    timestamp_match = re.search('\((.+?)\)', paragraph.text)
    if timestamp_match:
        timestamp.append(timestamp_match.group(1))
    
    corp_match_index = paragraph.text.find(")")+2
    if corp_match_index:
        text.append(paragraph.text[corp_match_index:])

briefing_data = pd.DataFrame(list(zip(timestamp, speaker, text)),
                                 columns = ['timestamp','speaker','text'])

# extract briefing date and add to df
date_string = briefing.find_all("div",{"class":"fl-rich-text"})[0].text
if date_string:
    briefing_date = datetime.strptime(date_string, '%b %d, %Y')
    
briefing_data.insert(0,'date',briefing_date)

briefing_data

,date,timestamp,speaker,text
0,2020-04-03,00:00,Donald Trump,"With Kevin McCarthy, future speaker of the hou..."
1,2020-04-03,00:27,Kevin McCarthy,Well thank you Mr. President and I would like ...
2,2020-04-03,01:20,Donald Trump,Great job you’re doing. Thank you.
3,2020-04-03,01:22,Kevin McCarthy,Thank you.
4,2020-04-03,01:22,Donald Trump,You go ahead. I’ll talk to you later.
...,...,...,...,...
229,2020-04-03,01:15:34,Speaker 26,Do you think every state in this country shoul...
230,2020-04-03,01:15:39,Donald Trump,No. I think a lot of people cheat with mail in...
231,2020-04-03,01:16:11,Donald Trump,It should be you go to a booth and you proudly...
232,2020-04-03,01:16:39,Speaker 27,Do you have an update on the domestic travel r...


In [63]:
filename = briefing_date.strftime("%Y-%m-%d")+("_tfb.csv")


In [65]:
filename = "../data/" + briefing_date.strftime("%Y-%m-%d")+("_tfb.csv")
filename

'../data/2020-04-03_tfb.csv'

In [66]:
briefing_data.to_csv(filename,index=False)